# Define Dynamic System Model

In [1]:
from sympy import *

# states, inputs, and process noise
X=Matrix(symbols('X[0] X[1] X[2] X[3] X[4] X[5] X[6] X[7] X[8] X[9] X[10] X[11] X[12] X[13] X[14]'))
U=Matrix(symbols('U[0] U[1] U[2] U[3] U[4] U[5]'))
W=Matrix(symbols('W[0] W[1] W[2] W[3] W[4] W[5]'))

# time step
dt=symbols('dt')

# continuous dynamics:
g = 9.81

x,y,z,vx,vy,vz,phi,theta,psi,lx,ly,lz,lp,lq,lr = X
ax,ay,az,p,q,r = U
wx,wy,wz,wp,wq,wr = W

Rx = Matrix([[1, 0, 0], [0, cos(phi), -sin(phi)], [0, sin(phi), cos(phi)]])
Ry = Matrix([[cos(theta), 0, sin(theta)],[0, 1, 0],[-sin(theta), 0, cos(theta)]])
Rz = Matrix([[cos(psi), -sin(psi), 0],[sin(psi), cos(psi), 0], [0, 0, 1]])
R = Rz*Ry*Rx

# body velocity
vbx,vby,vbz = R.T*Matrix([vx,vy,vz])

# only when the drone is above the ground (z-axis pointing down)
a_NED = R*Matrix([ax-lx-wx,ay-ly-wy,az-lz-wz])

f_continuous = Matrix([
    vx,
    vy,
    vz,
    a_NED[0],
    a_NED[1],
    a_NED[2] + g,
    (p-lp-wp) + (q-lq-wq)*sin(phi)*tan(theta) + (r-lr-wr)*cos(phi)*tan(theta),
    (q-lq-wq)*cos(phi) - (r-lr-wr)*sin(phi),
    (q-lq-wq)*sin(phi)/cos(theta) + (r-lr-wr)*cos(phi)/cos(theta),
    0,0,0,0,0,0
])

# discretized dynamics:
f = X + f_continuous*dt

# output function (measurement model):
use_phi, use_theta, use_psi = symbols('ekf_use_phi ekf_use_theta ekf_use_psi') # 0 or 1
h = Matrix([x,y,z,use_phi*phi,use_theta*theta,use_psi*psi])

# matrices:
F = f.jacobian(X)
L = f.jacobian(W)
H = h.jacobian(X)

# substitute W with 0
f = f.subs([(w,0) for w in W])
F = F.subs([(w,0) for w in W])
L = L.subs([(w,0) for w in W])
H = H.subs([(w,0) for w in W])

# extra matrices
symmetric_indexing = lambda i,j: int(i*(i+1)/2+j) if i>=j else int(j*(j+1)/2+i)
P = Matrix([[symbols(f'P[{symmetric_indexing(i,j)}]') for j in range(len(X))] for i in range(len(X))])
Q = Matrix([[symbols(f'Q[{i}]') if i==j else '0' for j in range(len(W))] for i in range(len(W))])
R = Matrix([[symbols(f'R[{i}]') if i==j else '0' for j in range(len(h))] for i in range(len(h))])
Z = Matrix([symbols(f'Z[{i}]') for i in range(len(h))])

# Generate EKF code

In [2]:
# function for inverting matrix with symbolic elements that is quicker
def invert_function(num):
    M = Matrix([[symbols(f'M[{i}{j}]') for j in range(num)] for i in range(num)])
    M_inv = M.inverse_LU()
    return lambda M_: M_inv.subs({M[i,j]:M_[i,j] for i in range(num) for j in range(num)})

In [3]:
# from sympy import symbols, sin
from sympy.codegen.ast import CodeBlock, Assignment

# EKF equations from: https://en.wikipedia.org/wiki/Extended_Kalman_filter: Non-additive noise formulation and equations

# PREDICTION STEP
Xpred = f
Ppred = F*P*F.T + L*Q*L.T

# UPDATE STEP
S = H*P*H.T + R
inv__ = invert_function(S.shape[0])
K = P*H.T*inv__(S)
Xup = X + K*(Z - h)
Pup = (eye(len(X)) - K*H)*P


# assignments
Xpred_assigments = [Assignment(symbols(f'X_new[{i}]'), Xpred[i]) for i in range(len(X))]
Ppred_assigments = [Assignment(symbols(f'P_new[{symmetric_indexing(i,j)}]'), Ppred[i,j]) for i in range(len(X)) for j in range(len(X)) if i >= j] # only the lower diagonal will be calculated

Xup_assigments = [Assignment(symbols(f'X_new[{i}]'), Xup[i]) for i in range(len(X))]
Pup_assigments = [Assignment(symbols(f'P_new[{symmetric_indexing(i,j)}]'), Pup[i,j]) for i in range(len(X)) for j in range(len(X)) if i >= j] # only the lower diagonal will be calculated

# PREDICTION STEP
print('PREDICTION:')
# code generation
prediction_code = CodeBlock(*Xpred_assigments, *Ppred_assigments)
# common subexpression elimination with tmp variables
print('CSE')
prediction_code = prediction_code.cse(symbols=(symbols(f'tmp[{i}]') for i in range(10000)))
# simplify
print('SIMPLIFY')
prediction_code = prediction_code.simplify()
# count number of tmp variables
prediction_code_N_tmps = len([s for s in prediction_code.left_hand_sides if s.name.startswith('tmp')])
# generate C code
print('CCODE')
prediction_code = ccode(prediction_code)

# UPDATE STEP
print('UPDATE:')
# code generation
update_code = CodeBlock(*Xup_assigments, *Pup_assigments)
# common subexpression elimination with tmp variables
print('CSE')
update_code = update_code.cse(symbols=(symbols(f'tmp[{i}]') for i in range(10000)))
# simplify
print('SIMPLIFY')
update_code = update_code.simplify()
# count number of tmp variables
update_code_N_tmps = len([s for s in update_code.left_hand_sides if s.name.startswith('tmp')])
# generate C code
print('CCODE')
update_code = ccode(update_code)


PREDICTION:
CSE
SIMPLIFY
CCODE
UPDATE:
CSE
SIMPLIFY
CCODE


In [6]:
# replace sin, cos, tan, pow with sinf, cosf, tanf, powf
prediction_code = prediction_code.replace('sin(', 'sinf(')
prediction_code = prediction_code.replace('cos(', 'cosf(')
prediction_code = prediction_code.replace('tan(', 'tanf(')
prediction_code = prediction_code.replace('pow(', 'powf(')
update_code = update_code.replace('sin(', 'sinf(')
update_code = update_code.replace('cos(', 'cosf(')
update_code = update_code.replace('tan(', 'tanf(')
update_code = update_code.replace('pow(', 'powf(')

print('Prediction code:')
print(prediction_code)
print('Update code:')
print(update_code)

Prediction code:
tmp[0] = cosf(X[8]);
tmp[1] = cosf(X[7]);
tmp[2] = U[0] - X[9];
tmp[3] = tmp[1]*tmp[2];
tmp[4] = U[2] - X[11];
tmp[5] = sinf(X[6]);
tmp[6] = sinf(X[8]);
tmp[7] = tmp[5]*tmp[6];
tmp[8] = sinf(X[7]);
tmp[9] = cosf(X[6]);
tmp[10] = tmp[0]*tmp[9];
tmp[11] = tmp[10]*tmp[8] + tmp[7];
tmp[12] = U[1] - X[10];
tmp[13] = tmp[6]*tmp[9];
tmp[14] = tmp[0]*tmp[5];
tmp[15] = -tmp[13] + tmp[14]*tmp[8];
tmp[16] = dt*(tmp[0]*tmp[3] + tmp[11]*tmp[4] + tmp[12]*tmp[15]);
tmp[17] = tmp[3]*tmp[6];
tmp[18] = tmp[10] + tmp[7]*tmp[8];
tmp[19] = tmp[14] - tmp[6]*tmp[8]*tmp[9];
tmp[20] = -tmp[19];
tmp[21] = tmp[2]*tmp[8];
tmp[22] = tmp[12]*tmp[1];
tmp[23] = tmp[1]*tmp[4];
tmp[24] = tanf(X[7]);
tmp[25] = U[4] - X[13];
tmp[26] = tmp[25]*tmp[5];
tmp[27] = U[5] - X[14];
tmp[28] = tmp[27]*tmp[9];
tmp[29] = tmp[25]*tmp[9];
tmp[30] = tmp[27]*tmp[5];
tmp[31] = 1.0/tmp[1];
tmp[32] = tmp[0]*tmp[1];
tmp[33] = dt*tmp[32];
tmp[34] = -tmp[15];
tmp[35] = dt*tmp[34];
tmp[36] = -tmp[11];
tmp[37] = dt*tmp[36];
tmp

# Write C code

In [7]:
# create c_code folder (if it doesn't exist)
import os
if not os.path.exists('c_code2'):
    os.makedirs('c_code2')
    
header = f'''
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
// This file is automatically generated in "Generate Kalman Filter C Code.ipynb" from https://github.com/tudelft/kalman_filter   //
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

#ifndef EKF_CALC_H
#define EKF_CALC_H

#include <stdint.h>

#define N_STATES {len(X)}
#define N_INPUTS {len(U)}
#define N_MEASUREMENTS {len(h)}

// set to 1 to use phi, theta, psi measurements
float ekf_use_phi;
float ekf_use_theta;
float ekf_use_psi;

// getters
float* ekf_get_X();     // get state vector
float* ekf_get_P();     // get covariance matrix (lower diagonal)

#define ekf_P_index(i,j) ((i>=j) ? ekf_get_P()[i*(i+1)/2+j] : ekf_get_P()[j*(j+1)/2+i])
#define ekf_X_index(i) ekf_get_X()[i]

// setters
void ekf_set_Q(float Q[N_INPUTS]);                    // set process noise covariance matrix diagonal
void ekf_set_R(float R[N_MEASUREMENTS]);              // set measurement noise covariance matrix diagonal
void ekf_set_X(float X0[N_STATES]);                   // set state vector
void ekf_set_P_diag(float P_diag[N_STATES]);          // set covariance matrix diagonal
void ekf_set_P(float P0[N_STATES*(N_STATES+1)/2]);    // set covariance matrix (lower diagonal)

// prediction and update functions
void ekf_predict(float U[N_INPUTS], float dt);
void ekf_update(float Z[N_MEASUREMENTS]);


#endif // EKF_CALC_H
'''

prediction_code_ = prediction_code.replace('\n','\n\t')
update_code_ = update_code.replace('\n','\n\t')

source = f'''
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
// This file is automatically generated in "Generate Kalman Filter C Code.ipynb" from https://github.com/tudelft/kalman_filter   //
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

#include "ekf_calc.h"
#include <math.h>

float ekf_use_phi = 1;
float ekf_use_theta = 1;
float ekf_use_psi = 1;

float ekf_Q[N_INPUTS];         // Kalman filter process noise covariance matrix (diagonal)
float ekf_R[N_MEASUREMENTS];   // Kalman filter measurement noise covariance matrix (diagonal)

// state
float ekf_X[N_STATES];
float ekf_X_new[N_STATES];

// covariance matrix (lower diagonal) P[i,j] = P_lower_diagonal[i*(i+1)/2+j] (if i>=j)
float ekf_P_lower_diagonal[N_STATES*(N_STATES+1)/2];
float ekf_P_lower_diagonal_new[N_STATES*(N_STATES+1)/2];

// temporary variables
float tmp[{max(prediction_code_N_tmps,update_code_N_tmps)}];

// pointers
float *X = ekf_X;
float *X_new = ekf_X_new;

float *P = ekf_P_lower_diagonal;
float *P_new = ekf_P_lower_diagonal_new;

// renaming
float *Q = ekf_Q;
float *R = ekf_R;

// pointer for swapping
float *swap_ptr;

float* ekf_get_X() {{
    return X;
}}

float* ekf_get_P() {{
    return P;
}}

void ekf_set_Q(float Q[N_INPUTS]) {{
    for (int i=0; i<N_INPUTS; i++) {{
        ekf_Q[i] = Q[i];
    }}
}}

void ekf_set_R(float R[N_MEASUREMENTS]) {{
    for (int i=0; i<N_MEASUREMENTS; i++) {{
        ekf_R[i] = R[i];
    }}
}}

void ekf_set_X(float X0[N_STATES]) {{
    for (int i=0; i<N_STATES; i++) {{
        X[i] = X0[i];
    }}
}}

void ekf_set_P_diag(float P_diag[N_STATES]) {{
    // set P to zeros
    for (int i=0; i<N_STATES*(N_STATES+1)/2; i++) {{
        P[i] = 0.;
    }}
    // set diagonal
    for (int i=0; i<N_STATES; i++) {{
        P[i*(i+1)/2+i] = P_diag[i];
    }}
}}

void ekf_set_P(float P0[N_STATES*(N_STATES+1)/2]) {{
    for (int i=0; i<N_STATES*(N_STATES+1)/2; i++) {{
        P[i] = P0[i];
    }}
}}

void ekf_predict(float U[N_INPUTS], float dt) {{
    // PREDICTION STEP X_new, P_new = ...
    {prediction_code_}

    // swap X, X_new and P, P_new pointers
    swap_ptr = X;
    X = X_new;
    X_new = swap_ptr;

    swap_ptr = P;
    P = P_new;
    P_new = swap_ptr;
}}

void ekf_update(float Z[N_MEASUREMENTS]) {{
    // UPDATE STEP X_new, P_new = ...
    {update_code_}

    // swap X, X_new and P, P_new pointers
    swap_ptr = X;
    X = X_new;
    X_new = swap_ptr;

    swap_ptr = P;
    P = P_new;
    P_new = swap_ptr;
}}
'''

with open('c_code2/ekf_calc.h','w') as file:
    file.write(header)
with open('c_code2/ekf_calc.c','w') as file:
    file.write(source)
    

# Compile C Code

In [8]:
import os
import subprocess
import ctypes
import numpy as np
import importlib
importlib.reload(ctypes)
importlib.reload(subprocess)

# Compile C code and create library

# https://cu7ious.medium.com/how-to-use-dynamic-libraries-in-c-46a0f9b98270
path = os.path.abspath('c_code2')
# Remove old library
subprocess.call('rm *.so', shell=True, cwd=path)
# Compile object files
subprocess.call('gcc -fPIC -c *.c', shell=True, cwd=path)
# Create library
subprocess.call('gcc -shared -Wl,-soname,libtools.so -o libtools.so *.o', shell=True, cwd=path)
# Remove object files
# subprocess.call('rm *.o', shell=True, cwd=path)

lib_path = os.path.abspath("c_code2/libtools.so")
lib = ctypes.CDLL(lib_path)

# define argument types 
lib.ekf_set_Q.argtypes = [ctypes.POINTER(ctypes.c_float)]
lib.ekf_set_R.argtypes = [ctypes.POINTER(ctypes.c_float)]
lib.ekf_set_X.argtypes = [ctypes.POINTER(ctypes.c_float)]
lib.ekf_set_P_diag.argtypes = [ctypes.POINTER(ctypes.c_float)]

lib.ekf_get_X.restype = ctypes.POINTER(ctypes.c_float)
lib.ekf_get_P.restype = ctypes.POINTER(ctypes.c_float)

lib.ekf_predict.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.c_float]
lib.ekf_update.argtypes = [ctypes.POINTER(ctypes.c_float)]

rm: cannot remove '*.so': No such file or directory


# Test C code

In [10]:
import os
from analyze import *

# check for symbolic link to flight_data folder
if not os.path.islink("flight_data"):
    os.symlink("/home/robinferede/Git/analyze_flight_data/flight_data", "flight_data")
    
# load data
# path = 'flight_data/Sep4_5inch_icra_5inch_20percent_randomized_3_3succesfull_runs.csv'
path = 'flight_data/Sep23_5inch_ground_exp_1.csv'
data = load_flight_data(path, new_format=True)

def trim_nn_active(data):
    indices = data['flightModeFlags'] > 8000
    data = {k: v[indices] for k, v in data.items() if isinstance(v, np.ndarray) and len(v) == len(indices)}
    data['t'] = data['t'] - data['t'][0]
    return data

data = trim_nn_active(data)

# cut data after the drone takes off z<-0.2
# start = np.where(data['z'] < -0.2)[0][0]
# data = {key: value[start:] for key, value in data.items() if isinstance(value, np.ndarray) and len(value) > start}

Loading flight_data/Sep23_5inch_ground_exp_1.csv
Logging frequency: 969.1348423012907


In [11]:
import sys
importlib.reload(sys.modules['analyze'])
from analyze import *

# animate
animate_data(data)

# plot thrust drag model
%matplotlib tk
p = fit_thrust_drag_model(data)
print('k_x:',p[0],'k_y:',p[1],'k_w:',p[2])

QObject::moveToThread: Current thread (0x62c29e2e2300) is not the object's thread (0x62c2a56be2d0).
Cannot move to target thread (0x62c29e2e2300)

QObject::moveToThread: Current thread (0x62c29e2e2300) is not the object's thread (0x62c2a56be2d0).
Cannot move to target thread (0x62c29e2e2300)

QObject::moveToThread: Current thread (0x62c29e2e2300) is not the object's thread (0x62c2a56be2d0).
Cannot move to target thread (0x62c29e2e2300)

QObject::moveToThread: Current thread (0x62c29e2e2300) is not the object's thread (0x62c2a56be2d0).
Cannot move to target thread (0x62c29e2e2300)

QObject::moveToThread: Current thread (0x62c29e2e2300) is not the object's thread (0x62c2a56be2d0).
Cannot move to target thread (0x62c29e2e2300)

QObject::moveToThread: Current thread (0x62c29e2e2300) is not the object's thread (0x62c2a56be2d0).
Cannot move to target thread (0x62c29e2e2300)

QObject::moveToThread: Current thread (0x62c29e2e2300) is not the object's thread (0x62c2a56be2d0).
Cannot move to tar

fitting thrust and drag model
k_x: -2.582789633164416e-06 k_y: -8.225556180825891e-05 k_w: -7.935965681405393e-05


In [12]:
# EKF INPUT: U = [ax, ay, az, p, q, r] (IMU)
U_data = np.array([
    data['ax'], data['ay'], data['az'], data['p'], data['q'], data['r']
]).T

# EKF MEASUREMENT: Z = [x, y, z, phi, theta, psi] (Optitrack)
Z_data = np.array([
    data['x_opti'], data['y_opti'], data['z_opti'], data['phi_opti'], data['theta_opti'], data['psi_opti']
]).T

t = data['t']

In [15]:
def run_ekf(t, X_init, P_init, U_data, Z_data, Q_diag, R_diag):
    lib.ekf_set_Q((ctypes.c_float*len(Q_diag))(*Q_diag))
    lib.ekf_set_R((ctypes.c_float*len(R_diag))(*R_diag))
    lib.ekf_set_X((ctypes.c_float*len(X_init))(*X_init))
    lib.ekf_set_P_diag((ctypes.c_float*len(P_init))(*P_init))

    # test ekf_get_state
    x = lib.ekf_get_X()
    x = np.array([x[i] for i in range(len(X))])

    # RUN EKF
    X_ekf = [X_init]
    # Pdiag_ekf = [P_init]

    for i in range(1, len(t)):
        # PREDICT
        lib.ekf_predict(
            (ctypes.c_float*len(U_data[i]))(*U_data[i]),
            ctypes.c_float(t[i]-t[i-1])
        )
        
        # UPDATE
        # only update when there is a new measurement
        if np.any(Z_data[i]-Z_data[i-1] != 0):            
            # fix yaw
            while Z_data[i,5]-Z_data[i-1,5] > np.pi:
                Z_data[i,5] -= 2*np.pi
            while Z_data[i,5]-Z_data[i-1,5] < -np.pi:
                Z_data[i,5] += 2*np.pi
            
            # update
            # lib.ekf_update(
            #     (ctypes.c_float*len(Z_data[i]))(*Z_data[i])
            # )    
            
        # get state
        x = lib.ekf_get_X()
        x = np.array([x[i] for i in range(len(X))])
        X_ekf.append(x)
    return np.array(X_ekf)

In [16]:
# INITIALIZE EKF
# initial state
X_init = np.zeros(len(X))
X_init[:3] = Z_data[0,:3]
X_init[8] = Z_data[0,5]

# initial covariance
P_init = np.array([1.]*len(X))
# P_init[-3:] = 0. # set covariance of gyro bias to 0
print(P_init)

# process noise covariance
Q_diag = np.array([.5, .5, .5, 0.1, 0.1, 0.1])

# measurement noise covariance
R_diag = np.array([0.001, 0.001, 0.001, 0.01, 0.01, 0.01])

X_ekf = run_ekf(t, X_init, P_init, U_data, Z_data, Q_diag, R_diag)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [19]:
# plot trajectory
%matplotlib tk
# Pdiag_ekf = np.array(Pdiag_ekf)
X_ekf = np.array(X_ekf)
opti_indices = [i for i in range(len(t)) if np.any(Z_data[i]-Z_data[i-1] != 0)]

# 5x3 subplots
fig, axs = plt.subplots(3, 3, figsize=(20,20), tight_layout=True, sharex=True, sharey='row')
axs = axs.flatten()

# position
axs[0].plot(t[opti_indices], data['x_opti'][opti_indices], label='x opti')
axs[0].plot(t, X_ekf[:,0], label='x ekf')
# axs[0].plot(t, data['ekf_x'], label='x ekf onboard')
axs[0].legend()


axs[1].plot(t[opti_indices], data['y_opti'][opti_indices], label='y opti')
axs[1].plot(t, X_ekf[:,1], label='y ekf')
# axs[1].plot(t, data['ekf_y'], label='y ekf onboard')
axs[1].legend()

axs[2].plot(t[opti_indices], data['z_opti'][opti_indices], label='z opti')
axs[2].plot(t, X_ekf[:,2], label='z ekf')
# axs[2].plot(t, data['ekf_z'], label='z ekf onboard')
axs[2].legend()

# velocity
axs[3].plot(t, X_ekf[:,3], label='vx ekf')
# axs[3].plot(t, data['ekf_vx'], label='vx ekf onboard')
axs[3].legend()

axs[4].plot(t, X_ekf[:,4], label='vy ekf')
# axs[4].plot(t, data['ekf_vy'], label='vy ekf onboard')
axs[4].legend()

axs[5].plot(t, X_ekf[:,5], label='vz ekf')
# axs[5].plot(t, data['ekf_vz'], label='vz ekf onboard')
axs[5].legend()

# attitude
rad_to_deg = 180/np.pi
axs[6].plot(t[opti_indices], rad_to_deg*data['phi_opti'][opti_indices], label='phi opti')
axs[6].plot(t, rad_to_deg*X_ekf[:,6], label='phi ekf')
# axs[6].plot(t, rad_to_deg*data['ekf_phi'], label='phi ekf onboard')
axs[6].legend()

axs[7].plot(t[opti_indices], rad_to_deg*data['theta_opti'][opti_indices], label='theta opti')
axs[7].plot(t, rad_to_deg*X_ekf[:,7], label='theta ekf')
# axs[7].plot(t, rad_to_deg*data['ekf_theta'], label='theta ekf onboard')
axs[7].legend()

axs[8].plot(t[opti_indices], rad_to_deg*data['psi_opti'][opti_indices], label='psi opti')
axs[8].plot(t, rad_to_deg*X_ekf[:,8], label='psi ekf')
# axs[8].plot(t, rad_to_deg*data['ekf_psi'], label='psi ekf onboard')
axs[8].legend()

# biases
# axs[9].plot(t, X_ekf[:,9], label='lx ekf')
# # axs[9].plot(t, data['ekf_acc_b_x'], label='lx ekf onboard')
# axs[9].legend()

# axs[10].plot(t, X_ekf[:,10], label='ly ekf')
# # axs[10].plot(t, data['ekf_acc_b_y'], label='ly ekf onboard')
# axs[10].legend()

# axs[11].plot(t, X_ekf[:,11], label='lz ekf')
# # axs[11].plot(t, data['ekf_acc_b_z'], label='lz ekf onboard')
# axs[11].legend()

# axs[12].plot(t, rad_to_deg*X_ekf[:,12], label='lp ekf')
# # axs[12].plot(t, rad_to_deg*data['ekf_gyro_b_x'], label='lp ekf onboard')
# axs[12].legend()

# axs[13].plot(t, rad_to_deg*X_ekf[:,13], label='lq ekf')
# # axs[13].plot(t, rad_to_deg*data['ekf_gyro_b_y'], label='lq ekf onboard')
# axs[13].legend()

# axs[14].plot(t, rad_to_deg*X_ekf[:,14], label='lr ekf')
# # axs[14].plot(t, rad_to_deg*data['ekf_gyro_b_z'], label='lr ekf onboard')
# axs[14].legend()

# still have tick labels for all subplots
for ax in axs:
    ax.label_outer()

plt.show()

In [42]:
plt.close('all')